In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
class PINN(tf.keras.Model):
    def __init__(self, layers, activation_function):
        super(PINN, self).__init__()
        self.layers_ = []
        self.scaling_layer = tf.keras.layers.Lambda(lambda x: 2.0*(x - lb)/(ub - lb) - 1.0)  # Scaling layer
        for layer_width in layers[:-1]:
            self.layers_.append(tf.keras.layers.Dense(layer_width, activation='tanh'))
        self.layers_.append(tf.keras.layers.Dense(layers[-1], activation=None))  # Output layer

    def call(self, x):
        x = self.scaling_layer(x)
        for layer in self.layers_:
            x = layer(x)
        return x

In [3]:
# Define f(x,t)
def f(x, t):
    return np.exp(-t) * np.sin(np.pi * x) * (-1 + np.pi**2)

In [4]:
# Define the PDE residual for the heat equation: u_t = u_xx + f(x,t)
def heat_residual(t, x, u, u_t, u_xx):
    f_val = tf.convert_to_tensor(f(x, t), dtype=tf.float32)
    return u_t - u_xx - f_val

In [5]:
# Loss function incorporating Neumann boundary conditions
def loss_fn(model, t_x, X_ic, u_ic, X_bc, g_bc_left, g_bc_right):
    t = t_x[:, 0:1]
    x = t_x[:, 1:2]

    # Initial condition residual
    u_pred_initial = model(X_ic)
    loss_ic = tf.reduce_mean(tf.square(u_pred_initial - u_ic))

    # Boundary condition residual (Neumann boundary conditions: u_x(0, t) = pi * exp(-t), u_x(1, t) = -pi * exp(-t))
    with tf.GradientTape(persistent=True) as tape_bc_left:
        tape_bc_left.watch(X_bc['left'])
        u_bc_left = model(X_bc['left'])
    u_x_bc_left = tape_bc_left.gradient(u_bc_left, X_bc['left'])[:, 1:2]
    loss_bc_left = tf.reduce_mean(tf.square(u_x_bc_left - g_bc_left))

    with tf.GradientTape(persistent=True) as tape_bc_right:
        tape_bc_right.watch(X_bc['right'])
        u_bc_right = model(X_bc['right'])
    u_x_bc_right = tape_bc_right.gradient(u_bc_right, X_bc['right'])[:, 1:2]
    loss_bc_right = tf.reduce_mean(tf.square(u_x_bc_right - g_bc_right))

    # PDE residual
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(t)
        tape.watch(x)
        u = model(tf.concat([t, x], axis=1))
        u_x = tape.gradient(u, x)
    u_t = tape.gradient(u, t)
    u_xx = tape.gradient(u_x, x)

    residual = heat_residual(t, x, u, u_t, u_xx)
    loss_pde = tf.reduce_mean(tf.square(residual))

    return loss_pde + loss_ic + loss_bc_left + loss_bc_right

In [6]:
# Training function
def train_model(model, X_train, X_ic, u_ic, X_bc, g_bc_left, g_bc_right, epochs=1000, learning_rate=1e-3):
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            loss = loss_fn(model, X_train, X_ic, u_ic, X_bc, g_bc_left, g_bc_right)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        if epoch % 100 == 0:
            print(f'Epoch: {epoch}, Loss: {loss.numpy()}')


In [7]:
# Exact solution (for comparison if available)
def exact_solution(t, x):
    return np.exp(-t) * np.sin(np.pi * x)

In [8]:
# Define grid points for training
N = 100
T_max = 1.0
tspace = np.linspace(0, T_max, N)
xspace = np.linspace(0, 1, N)
T, X = np.meshgrid(tspace, xspace)
Xgrid = np.vstack([T.flatten(), X.flatten()]).T

# Convert grid to TensorFlow tensor
X_train = tf.convert_to_tensor(Xgrid, dtype=tf.float32)

# Initial condition: u(0, x) = sin(pi * x)
X_ic = np.vstack([np.zeros(N), xspace]).T  # t=0 for all x
u_ic = np.sin(np.pi * X_ic[:, 1:2])        # sin(pi * x)
X_ic = tf.convert_to_tensor(X_ic, dtype=tf.float32)
u_ic = tf.convert_to_tensor(u_ic, dtype=tf.float32)

# Neumann boundary conditions: u_x(0, t) = pi * exp(-t), u_x(1, t) = -pi * exp(-t)
g_bc_left = tf.reshape(tf.convert_to_tensor(np.pi * np.exp(-tspace), dtype=tf.float32), (N, 1))
g_bc_right = tf.reshape(tf.convert_to_tensor(-np.pi * np.exp(-tspace), dtype=tf.float32), (N, 1))

# Boundary points for left and right (for Neumann BCs)
X_bc_left = np.vstack([tspace, np.zeros(N)]).T   # x=0 for all t
X_bc_right = np.vstack([tspace, np.ones(N)]).T   # x=1 for all t
X_bc = {'left': tf.convert_to_tensor(X_bc_left, dtype=tf.float32),
        'right': tf.convert_to_tensor(X_bc_right, dtype=tf.float32)}

In [9]:
# Build the model
layers = [2, 50, 50, 50, 1]  # Neural network architecture
model = PINN(layers)

# Train the model
train_model(model, X_train, X_ic, u_ic, X_bc, g_bc_left, g_bc_right, epochs=2000, learning_rate=1e-3)

TypeError: PINN.__init__() missing 1 required positional argument: 'activation_function'

In [ ]:
# Predictions
upred = model(X_train)
U_pred = upred.numpy().reshape(N, N)

In [ ]:
# Exact solution for comparison (if available)
U_exact = exact_solution(T, X)

In [ ]:
# Plot predicted vs exact solution
fig = plt.figure(figsize=(12, 6))

# Plot predicted solution
ax1 = fig.add_subplot(121, projection='3d')
ax1.plot_surface(T, X, U_pred, cmap='viridis')
ax1.set_title('Predicted Solution $u(t, x)$')
ax1.set_xlabel('$t$')
ax1.set_ylabel('$x$')
ax1.set_zlabel('$u(t, x)$')

# Plot exact solution
ax2 = fig.add_subplot(122, projection='3d')
ax2.plot_surface(T, X, U_exact, cmap='viridis')
ax2.set_title('Exact Solution $u(t, x) = e^{-t} \sin(\\pi x)$')
ax2.set_xlabel('$t$')
ax2.set_ylabel('$x$')
ax2.set_zlabel('$u(t, x)$')

plt.show()